In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk import word_tokenize,sent_tokenize,pos_tag_sents,pos_tag
import time

## 1. Read data.

In [2]:
DATADIR=os.getenv('DATADIR')
print(DATADIR)
DATAPATH = os.path.join(DATADIR, 'labelled_level2.csv.gz')

/Users/felisialoukou/Documents/govuk-taxonomy-supervised-learning/data


In [3]:
data = pd.read_csv(DATAPATH,compression='gzip')
data.shape

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(173560, 21)

In [4]:
data.head()

,base_path,content_id,description,details,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,...,body,combined_text,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon
0,/student-finance-register-login,e57daef4-5eb5-431c-b0ad-14119ab0355f,your student finance online account - check pa...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance login,...,,student finance login your student finance onl...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
1,/student-finance-calculator,434b6eb5-33c8-4300-aba3-f5ead58600b8,student finance calculator - get a quick estim...,{'introductory_paragraph': '<p>This calculator...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,smartanswers,student finance calculator,...,,student finance calculator student finance cal...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
2,/apply-online-for-student-finance,83155b50-418e-437c-9389-cf0e1302900f,apply online for student finance - online appl...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,apply online for student finance,...,,apply online for student finance apply online ...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
3,/student-finance,d38bafd3-2c46-4be2-b50b-50c2ba7d30ed,student finance - student loans or student gra...,"{'parts': [{'title': 'Overview', 'slug': 'over...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance,...,,student finance student finance - student loan...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
4,/apply-for-student-finance,06017464-d3e2-4a4c-8bef-250eff0de7e4,how to apply for student finance - when to app...,"{'parts': [{'title': 'How to apply', 'slug': '...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance: how to apply,...,,student finance: how to apply how to apply for...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN


### 1.2 UTILS.

In [11]:
data.columns

Index(['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'taxon_id', 'taxon_base_path', 'taxon_name', 'level1taxon',
       'level2taxon', 'level3taxon', 'level4taxon', 'level5taxon', 'title_len',
       'description_len', 'body_len', 'combined_text_len'],
      dtype='object')

In [12]:
def build_index(x):
    index_dict = {}
    index_dict['index'] = 0
    for i,elem in enumerate(x):
        index_dict[elem] = i+1
    return index_dict

In [13]:
ind = build_index(data.columns)
print(ind['combined_text'])

13


## 2. Counts

In [5]:
data['title_len'] = data['title'].map(lambda x: len(x.split()))
data['description_len'] = data['description'].map(lambda x: len(x.split()))
data['body_len'] = data['body'].map(lambda x: len(x.split()) if not isinstance(x,float) else 0)
data['combined_text_len'] = data['combined_text'].map(lambda x: len(x.split()))

In [6]:
len_cols = ['title_len','description_len','body_len','combined_text_len']

In [7]:
for col in len_cols:
    print(data[col].describe(),"\n=====")

count    173560.000000
mean          7.988482
std           3.214716
min           1.000000
25%           6.000000
50%           8.000000
75%          10.000000
max          38.000000
Name: title_len, dtype: float64 
=====
count    173560.000000
mean         17.732179
std           6.971012
min           0.000000
25%          13.000000
50%          18.000000
75%          21.000000
max         149.000000
Name: description_len, dtype: float64 
=====
count    173560.000000
mean        331.267758
std         612.127736
min           0.000000
25%          38.000000
50%         118.000000
75%         396.000000
max       31129.000000
Name: body_len, dtype: float64 
=====
count    173560.000000
mean        356.988419
std         612.851116
min           2.000000
25%          63.000000
50%         144.000000
75%         423.000000
max       31145.000000
Name: combined_text_len, dtype: float64 
=====


## 3. POS stuff.

In [8]:
data["combined_text"].iloc[0]

'student finance login your student finance online account - check payment dates track an application change details reset password find customer reference numbers  '

In [34]:
tokenized = word_tokenize(data["combined_text"].iloc[0])
tokenized

['student',
 'finance',
 'login',
 'your',
 'student',
 'finance',
 'online',
 'account',
 '-',
 'check',
 'payment',
 'dates',
 'track',
 'an',
 'application',
 'change',
 'details',
 'reset',
 'password',
 'find',
 'customer',
 'reference',
 'numbers']

In [10]:
data["combined_text"].values.shape

(173560,)

In [17]:
def reduce_text(x):
    keep_text =""
    sent_pos = pos_tag(word_tokenize(x))
    for word,tag in sent_pos:
        if tag.startswith("NN") or tag.startswith("VB") or tag.startswith("JJ"):
            keep_text += word + " "
    return keep_text.strip()

In [18]:
data['reduced_combined'] = data['combined_text'].map(reduce_text2)

In [26]:
data['combined_text'].iloc[100000]

"bittersweet' return from sangin for royal marines the first royal marines from 40 commando returned home yesterday from a six-month deployment to sangin in helmand province considered one of the most dangerous places for foreign troops in afghanistan. around 600 marines from 40 commando part of 3 commando brigade deployed to sangin in april 2010 under the command of 4th mechanized brigade on operation herrick 12. they are the last british unit to serve in the area having handed over authority to us forces in the last few weeks. see related news for more on this. the first of the marines to return home were from the 114-strong charlie company. they arrived at their home base of norton manor camp near taunton in somerset yesterday wednesday 6 october 2010 where they were greeted by more than 300 friends and family members. the homecoming was described as a ‘bittersweet’ moment by officers who recognised that the joy of returning to friends and family was tempered by the loss the unit ha

In [25]:
data['reduced_combined'].iloc[100000]

'bittersweet return sangin royal marines first royal marines commando returned home yesterday six-month deployment sangin helmand province considered dangerous places foreign troops afghanistan marines commando part commando brigade deployed sangin april command mechanized brigade operation herrick are last british unit serve area having handed authority forces last few weeks see related news more first marines return home were 114-strong charlie company arrived home base norton manor camp taunton somerset yesterday wednesday october were greeted more friends family members homecoming was described ‘ bittersweet ’ moment officers recognised joy returning friends family was tempered loss unit had experienced tour major duncan forbes officer commanding charlie company said ’ s bittersweet day being people battle group were killed were royal marines sacrifice was made vain did amazing job day showing bravery leadership is humbling leaders look ’ s sad day families have got husbands coming

In [38]:
data.columns

Index(['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'taxon_id', 'taxon_base_path', 'taxon_name', 'level1taxon',
       'level2taxon', 'level3taxon', 'level4taxon', 'level5taxon', 'title_len',
       'description_len', 'body_len', 'combined_text_len', 'reduced_combined'],
      dtype='object')

In [30]:
OUTPUT_PATH = os.path.join(DATADIR, 'labelled_level2_enriched_world_filtered.csv.gz')
OUTPUT_PATH

'/Users/felisialoukou/Documents/govuk-taxonomy-supervised-learning/data/labelled_level2_enriched_world_filtered.csv.gz'

In [31]:
data.to_csv(OUTPUT_PATH,index=False,compression='gzip')

### Doublecheck.

In [48]:
data_produced = pd.read_csv(OUTPUT_PATH,compression='gzip')

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
data_produced.head()

,base_path,content_id,description,details,document_type,first_published_at,locale,primary_publishing_organisation,publishing_app,title,...,combined_text,reduced_combined,taxon_id,taxon_base_path,taxon_name,level1taxon,level2taxon,level3taxon,level4taxon,level5taxon
0,/student-finance-register-login,e57daef4-5eb5-431c-b0ad-14119ab0355f,your student finance online account - check pa...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance login,...,student finance login your student finance onl...,student finance login student finance online a...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
1,/student-finance-calculator,434b6eb5-33c8-4300-aba3-f5ead58600b8,student finance calculator - get a quick estim...,{'introductory_paragraph': '<p>This calculator...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,smartanswers,student finance calculator,...,student finance calculator student finance cal...,student finance calculator student finance cal...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
2,/apply-online-for-student-finance,83155b50-418e-437c-9389-cf0e1302900f,apply online for student finance - online appl...,{'will_continue_on': 'the Student Finance Engl...,transaction,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,apply online for student finance,...,apply online for student finance apply online ...,online student finance apply online student fi...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
3,/student-finance,d38bafd3-2c46-4be2-b50b-50c2ba7d30ed,student finance - student loans or student gra...,"{'parts': [{'title': 'Overview', 'slug': 'over...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance,...,student finance student finance - student loan...,student finance student finance student loans ...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN
4,/apply-for-student-finance,06017464-d3e2-4a4c-8bef-250eff0de7e4,how to apply for student finance - when to app...,"{'parts': [{'title': 'How to apply', 'slug': '...",guide,2016-02-29T09:24:10.000+00:00,en,NaN,publisher,student finance: how to apply,...,student finance: how to apply how to apply for...,student finance apply apply student finance ap...,64aa6eec-48b5-481d-9131-9c8b6326eea1,/education/student-grants-bursaries-scholarships,"Student grants, bursaries and scholarships","Education, training and skills",Funding and finance for students,"Student grants, bursaries and scholarships",NaN,NaN


In [50]:
data_produced.columns

Index(['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'reduced_combined', 'taxon_id', 'taxon_base_path', 'taxon_name',
       'level1taxon', 'level2taxon', 'level3taxon', 'level4taxon',
       'level5taxon'],
      dtype='object')

In [40]:
data_produced.drop(['title_len',
       'description_len', 'body_len', 'combined_text_len'],axis=1,inplace=True)

In [45]:
data_produced = data_produced[['base_path', 'content_id', 'description', 'details', 'document_type',
       'first_published_at', 'locale', 'primary_publishing_organisation',
       'publishing_app', 'title', 'document_type_gp', 'body', 'combined_text',
       'reduced_combined',
       'taxon_id', 'taxon_base_path', 'taxon_name', 'level1taxon',
       'level2taxon', 'level3taxon', 'level4taxon', 'level5taxon']]

In [47]:
data_produced.to_csv(OUTPUT_PATH,index=False,compression='gzip')

### 3.2 Using Stanford pos_tagger
* Should switch to coreNLP
* Not that much of a difference, VBs still getting confused with NNs.
* Takes a really long time to run, if using Java, opt for this.

In [13]:
# StanfordPOSTagger
from nltk.tag.stanford import StanfordPOSTagger
stanford_dir = '/Users/felisialoukou/Downloads/stanford-postagger-full-2017-06-09/'
modelfile = stanford_dir + 'models/english-bidirectional-distsim.tagger'
jarfile = stanford_dir + 'stanford-postagger.jar'

st = StanfordPOSTagger(model_filename=modelfile, path_to_jar=jarfile)

/Users/felisialoukou/.pyenv/versions/3.4.6/lib/python3.4/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [16]:
tokenized[0]

'student finance login your student finance online account - check payment dates track an application change details reset password find customer reference numbers sign in or register for student finance online.'

In [20]:
st.tag(tokenized[0].split())

[('student', 'NN'),
 ('finance', 'NN'),
 ('login', 'NN'),
 ('your', 'PRP$'),
 ('student', 'NN'),
 ('finance', 'NN'),
 ('online', 'NN'),
 ('account', 'NN'),
 ('-', ':'),
 ('check', 'NN'),
 ('payment', 'NN'),
 ('dates', 'NNS'),
 ('track', 'VBP'),
 ('an', 'DT'),
 ('application', 'NN'),
 ('change', 'NN'),
 ('details', 'NNS'),
 ('reset', 'VBP'),
 ('password', 'NN'),
 ('find', 'VBP'),
 ('customer', 'NN'),
 ('reference', 'NN'),
 ('numbers', 'NNS'),
 ('sign', 'VBP'),
 ('in', 'IN'),
 ('or', 'CC'),
 ('register', 'NN'),
 ('for', 'IN'),
 ('student', 'NN'),
 ('finance', 'NN'),
 ('online.', 'NN')]